In [1]:
# %pip install pyspark py4j

In [2]:
# %pip install h5py

In [3]:
# %pip install tqdm

In [4]:
# %pip install torch

In [5]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

import torch

# from pyspark.sql import SparkSession
# from pyspark.sql import Row
import pickle
import h5py

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [6]:
# spark = SparkSession.builder\
#         .master("local[*]")\
#         .appName('PySpark')\
#         .getOrCreate()

## Загружаем все признаки и джойним их с маппером

In [7]:
ecpf6_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/Дескрипторы rdkit/ecfp6_feature.pickle'
maccs_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/Дескрипторы rdkit/maccs_feature.pickle'
rdkit_2d_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/Дескрипторы rdkit/rdkit_2d_feature.pickle'
graph2vec_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/graph2vec/graph2vec_feture.pickle'
mol2vec_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/mol2vec/mol2vec_feature.pickle'
roBERTa_path = '/home/jupyter/datasphere/project/Расчёт дискрипторов/roBERTa/roBERTa_feture.pickle'

In [8]:
ecpf6_df = pd.read_pickle(ecpf6_path)
ecpf6_df.drop_duplicates(subset = ['smiles'], inplace = True)
ecpf6_df.columns = ['ecpf6_'+i if i != 'smiles' else 'smiles' for i in ecpf6_df.columns]
# ecpf6_df = spark.createDataFrame(ecpf6_df.iloc[:100])

In [9]:
maccs_df = pd.read_pickle(maccs_path)
maccs_df.drop_duplicates(subset = ['smiles'], inplace = True)
maccs_df.columns = ['maccs_'+i if i != 'smiles' else 'smiles' for i in maccs_df.columns]
# maccs_df = spark.createDataFrame(maccs_df.iloc[:100])

In [10]:
rdkit_2d_df = pd.read_pickle(rdkit_2d_path)
rdkit_2d_df.drop_duplicates(subset = ['smiles'], inplace = True)
rdkit_2d_df.columns = ['rdkit_2d_'+i if i != 'smiles' else 'smiles' for i in rdkit_2d_df.columns]
# rdkit_2d_df = spark.createDataFrame(rdkit_2d_df.iloc[:100])

In [11]:
graph2vec_df = pd.read_pickle(graph2vec_path)
graph2vec_df.columns = ['graph2vec_'+str(i) if str(i) != 'smiles' else 'smiles' for i in graph2vec_df.columns]
# graph2vec_df = spark.createDataFrame(graph2vec_df.iloc[:100])

In [12]:
mol2vec_df = pd.read_pickle(mol2vec_path)
mol2vec_df.columns = ['mol2vec_'+str(i) if str(i) != 'smiles' else 'smiles' for i in mol2vec_df.columns]
# mol2vec_df = spark.createDataFrame(mol2vec_df.iloc[:100])

In [13]:
roBERTa_df = pd.read_pickle(roBERTa_path)
roBERTa_df.columns = ['roBERTa_'+str(i) if str(i) != 'smiles' else 'smiles' for i in roBERTa_df.columns]
# roBERTa_df = spark.createDataFrame(roBERTa_df.iloc[:100])

In [14]:
mapper = pd.read_table('/home/jupyter/datasphere/project/Расчёт дискрипторов/mapper_moleculares.tsv')
# mapper = spark.createDataFrame(mapper)

In [15]:
reaction = pd.read_table('/home/jupyter/datasphere/project/reaction_preparation.tsv')
# reaction = spark.createDataFrame(reaction)

## Джойним таблички

In [16]:
df_res = mapper.copy()

for df in tqdm([rdkit_2d_df, maccs_df, graph2vec_df, mol2vec_df, roBERTa_df, ecpf6_df]):
    df_res = pd.merge(df_res, df, left_on = 'prepare_mol', right_on = 'smiles', how = 'left')
    df_res.drop('smiles', axis = 1, inplace = True)
    
del ecpf6_df, rdkit_2d_df, maccs_df, graph2vec_df, mol2vec_df, roBERTa_df

df_res.shape

100%|██████████| 6/6 [03:00<00:00, 30.16s/it]


(870765, 3679)

In [17]:
df_res.shape

(870765, 3679)

## Снижаем размерность данных

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [19]:
df_res.drop(['mol', 'prepare_mol'], axis = 1, inplace = True)

In [20]:
df_res = df_res.astype('float32')

In [21]:
for i in tqdm([j for j in df_res.columns if 'rdkit' in j]):
    q99 = df_res[i].quantile(0.99)
    df_res[i] = df_res[i].apply(lambda x: x if x <= q99 else q99)

100%|██████████| 210/210 [01:03<00:00,  3.30it/s]


In [22]:
scaler = StandardScaler()
df_res_scaler = scaler.fit_transform(df_res)
df_res_scaler.shape

(870765, 3677)

In [23]:
# import joblib
# joblib.dump(scaler, 'scaler.bin', compress=True)

In [24]:
# with open('./scaler.pkl','wb') as f:
#     pickle.dump(scaler, f)
# # with open('file/path/scaler.pkl','rb') as f:
# #     sc = pickle.load(f)

In [25]:
pca = PCA(0.95)  # сохраняем 95% доли объясненной дисперсии

# Обучаем PCA на данный X
df_res_reduced = pca.fit_transform(df_res_scaler)

In [26]:
# with open('./pca.pkl','wb') as f:
#     pickle.dump(pca, f)
# # with open('file/path/scaler.pkl','rb') as f:
# #     sc = pickle.load(f)

In [27]:
# joblib.dump(pca, 'pca.bin', compress=True)

In [28]:
df_res_reduced.shape

(870765, 1922)

In [29]:
pca_df = pd.DataFrame(df_res_reduced)
pca_df['mol'] = mapper['mol']
pca_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,mol
0,-12.580732,32.825262,-10.795974,2.129861,-3.549259,-2.693235,-0.421034,-3.272398,-8.256272,9.426660,7.497087,-9.835165,-1.112800,14.194845,-5.932571,8.774464,0.211985,0.798224,-5.021241,3.297688,1.523546,-1.123971,6.188115,-5.918546,-4.654739,5.082847,3.009504,-1.935585,-1.545462,-1.169225,-2.003127,1.730036,-4.437528,-4.983648,2.194239,2.037854,-2.494010,4.674025,2.922169,0.544350,...,0.029358,0.280455,-0.699837,-0.122484,0.496929,0.084986,-0.233996,-0.383175,-0.962394,-0.544596,1.165350,-0.029174,-0.307165,-0.080095,-0.263117,-0.310586,-0.840404,0.232310,-1.235119,0.383344,-0.350031,-0.763784,-0.076991,0.380183,0.637714,0.344091,0.313811,0.143428,-0.081613,0.674996,0.068928,0.433245,-0.240502,-0.216791,-0.020012,0.982769,0.718803,0.771975,-0.267873,O=[N+:22]([O-])[C:25]1=[C:26]([Cl:34])[C:27]([...
1,-17.310816,13.410915,-1.117257,-6.467861,29.689819,2.783287,8.484604,14.436696,-3.084702,0.728488,-3.591035,-0.751025,-18.360366,9.084855,7.963630,3.553094,2.017998,-4.453767,1.624839,-0.636075,0.220790,-2.046005,-0.506064,4.110558,-0.252478,1.641289,0.251554,-0.255471,-1.678605,0.144221,0.370166,-3.946183,0.447910,-0.587244,-1.815337,0.538314,-2.049750,1.973193,1.763855,-1.522597,...,-0.253068,0.297266,1.141953,0.660399,-0.446681,0.409131,-0.735960,0.552126,-0.155948,0.703875,0.168646,0.692044,-0.801220,-0.703427,-0.503206,0.392271,-0.091620,-0.346998,-0.429587,-0.276783,-0.014077,0.004421,-0.885714,-0.507088,0.517421,-1.150139,0.032562,-0.074911,-0.291660,-0.256323,0.117396,-0.298805,0.132705,0.713387,0.147233,-0.579008,-0.351454,0.566087,0.016375,C[Si-](C)(C)(F)F
2,-0.683197,20.998986,-2.793882,19.963518,20.969903,-6.755830,-2.290600,-3.446328,-4.964963,-0.920122,1.053040,-7.076007,3.628070,8.623472,4.239097,-1.842901,-0.305744,2.199301,-1.834375,-3.714469,6.896245,-0.519055,0.584445,-3.886087,3.260948,0.891281,-0.064003,4.703359,-3.065151,0.338065,-0.820268,-0.839649,-1.324685,1.575014,-1.233450,4.229714,4.826864,-2.630266,-1.153702,-2.006663,...,0.210774,0.512696,-0.072535,1.200272,-0.499747,0.411375,-0.552325,-0.623817,-0.016691,0.213244,-0.753056,0.049030,0.024117,-0.397610,-0.443780,0.670627,-0.646580,0.308547,0.041301,-0.291671,-0.868556,0.601215,-0.289495,-0.257117,-1.323369,-0.657632,0.345596,-0.599554,-0.137572,0.323551,-0.235368,-0.277566,-0.934612,-0.569062,-0.559647,-0.110479,0.261673,-1.307232,-0.266122,OC(=O)C1=CN2C=C(C(F)(F)F)[CH:19]=[C:14](Cl)[C:...
3,25.877104,-3.073434,5.325224,4.396501,15.780925,-1.647459,2.938136,-6.131052,2.542179,-1.394576,3.299235,-6.498743,-1.550451,1.437109,0.319132,1.193011,0.396962,2.226936,2.045064,-1.372769,0.588776,-1.274967,3.663682,-0.007115,0.110154,-3.971265,-0.555653,0.369941,-2.501207,-0.224877,-0.865648,1.037609,-3.807046,-1.913235,-0.617170,-0.367936,-0.952737,2.065924,-2.931635,-4.936853,...,-1.148173,-1.024835,0.321771,-0.357173,1.042426,0.436647,-0.193977,0.347655,0.208514,-0.586731,-0.691006,1.185702,0.409145,-2.550551,0.616470,0.480213,0.349182,-0.342722,1.907002,1.102293,0.458174,-0.909848,-0.131525,-1.388471,0.626571,-0.925222,2.790391,0.539964,-1.875945,0.667377,-0.392362,-0.462895,0.628174,1.924847,1.199006,-1.113347,-1.724282,-1.388951,-1.468666,CC1(C)[N:19]([C:24]([CH:25]([F:26])[F:27])=[O:...
4,19.973043,-9.691270,-2.542854,-1.903306,-3.660342,5.847983,4.150057,4.990500,2.908559,9.028728,-0.716515,8.659918,0.047819,-10.057276,2.159079,-0.310133,3.369029,2.009808,-1.942295,1.837200,7.544033,-4.985103,-0.644721,-2.436397,-8.846998,-3.676756,6.694883,4.116751,2.238149,0.136539,1.567481,-3.976882,-4.203535,0.611373,2.582165,-3.396374,0.995191,0.968641,0.060189,-2.792880,...,-1.129560,0.186216,0.542038,-0.867878,-0.176340,-0.972492,-0.808819,-0.418474,-1.1

In [30]:
# pca_df.to_pickle('feature_mol.pickle')

## Создаём трёхмерный массив |реакция|реагент|признаки|

In [31]:
reaction = reaction.drop(['interm_6', 'interm_7', 'interm_8', 'interm_9'], axis = 1)

In [32]:
def prepare_datasets(index):
    react = pd.DataFrame(list(reaction.iloc[index, :-1]), columns = ['mol'])
    react['index'] = index
    return react

In [33]:
df_react = [prepare_datasets(index) for index in tqdm(range(reaction.iloc[:10000].shape[0]))]

100%|██████████| 10000/10000 [00:04<00:00, 2320.50it/s]


In [34]:
df_react = pd.concat(df_react, axis = 0)
df_react.shape

(90000, 2)

In [35]:
pca_df.index = pca_df.mol
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,mol
mol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
O=[N+:22]([O-])[C:25]1=[C:26]([Cl:34])[C:27]([Cl:33])=N[C:29]([Cl:32])=[C:30]1[Cl:31],-12.580732,32.825262,-10.795974,2.129861,-3.549259,-2.693235,-0.421034,-3.272398,-8.256272,9.426660,7.497087,-9.835165,-1.112800,14.194845,-5.932571,8.774464,0.211985,0.798224,-5.021241,3.297688,1.523546,-1.123971,6.188115,-5.918546,-4.654739,5.082847,3.009504,-1.935585,-1.545462,-1.169225,-2.003127,1.730036,-4.437528,-4.983648,2.194239,2.037854,-2.494010,4.674025,2.922169,0.544350,...,0.029358,0.280455,-0.699837,-0.122484,0.496929,0.084986,-0.233996,-0.383175,-0.962394,-0.544596,1.165350,-0.029174,-0.307165,-0.080095,-0.263117,-0.310586,-0.840404,0.232310,-1.235119,0.383344,-0.350031,-0.763784,-0.076991,0.380183,0.637714,0.344091,0.313811,0.143428,-0.081613,0.674996,0.068928,0.433245,-0.240502,-0.216791,-0.020012,0.982769,0.718803,0.771975,-0.267873,O=[N+:22]([O-])[C:25]1=[C:26]([Cl:34])[C:27]([...
C[Si-](C)(C)(F)F,-17.310816,13.410915,-1.117257,-6.467861,29.689819,2.783287,8.484604,14.436696,-3.084702,0.728488,-3.591035,-0.751025,-18.360366,9.084855,7.963630,3.553094,2.017998,-4.453767,1.624839,-0.636075,0.220790,-2.046005,-0.506064,4.110558,-0.252478,1.641289,0.251554,-0.255471,-1.678605,0.144221,0.370166,-3.946183,0.447910,-0.587244,-1.815337,0.538314,-2.049750,1.973193,1.763855,-1.522597,...,-0.253068,0.297266,1.141953,0.660399,-0.446681,0.409131,-0.735960,0.552126,-0.155948,0.703875,0.168646,0.692044,-0.801220,-0.703427,-0.503206,0.392271,-0.091620,-0.346998,-0.429587,-0.276783,-0.014077,0.004421,-0.885714,-0.507088,0.517421,-1.150139,0.032562,-0.074911,-0.291660,-0.256323,0.117396,-0.298805,0.132705,0.713387,0.147233,-0.579008,-0.351454,0.566087,0.016375,C[Si-](C)(C)(F)F
OC(=O)C1=CN2C=C(C(F)(F)F)[CH:19]=[C:14](Cl)[C:15]2=N1,-0.683197,20.998986,-2.793882,19.963518,20.969903,-6.755830,-2.290600,-3.446328,-4.964963,-0.920122,1.053040,-7.076007,3.628070,8.623472,4.239097,-1.842901,-0.305744,2.199301,-1.834375,-3.714469,6.896245,-0.519055,0.584445,-3.886087,3.260948,0.891281,-0.064003,4.703359,-3.065151,0.338065,-0.820268,-0.839649,-1.324685,1.575014,-1.233450,4.229714,4.826864,-2.630266,-1.153702,-2.006663,...,0.210774,0.512696,-0.072535,1.200272,-0.499747,0.411375,-0.552325,-0.623817,-0.016691,0.213244,-0.753056,0.049030,0.024117,-0.397610,-0.443780,0.670627,-0.646580,0.308547,0.041301,-0.291671,-0.868556,0.601215,-0.289495,-0.257117,-1.323369,-0.657632,0.345596,-0.599554,-0.137572,0.323551,-0.235368,-0.277566,-0.934612,-0.569062,-0.559647,-0.110479,0.261673,-1.307232,-0.266122,OC(=O)C1=CN2C=C(C(F)(F)F)[CH:19]=[C:14](Cl)[C:...
CC1(C)[N:19]([C:24]([CH:25]([F:26])[F:27])=[O:28])[C@H:18]([CH2:29][F:30])[C@@H:17]([C:14]2=[CH:13][CH:12]=[C:11]([C:8]3=[CH:7][CH:6]=[C:5]([CH2:4][NH:3][CH2:1][CH3:2])[N:10]=[CH:9]3)[CH:16]=[CH:15]2)[O:21]1,25.877104,-3.073434,5.325224,4.396501,15.780925,-1.647459,2.938136,-6.131052,2.542179,-1.394576,3.299235,-6.498743,-1.550451,1.437109,0.319132,1.193011,0.396962,2.226936,2.045064,-1.372769,0.588776,-1.274967,3.663682,-0.007115,0.110154,-3.971265,-0.555653,0.369941,-2.501207,-0.224877,-0.865648,1.037609,-3.807046,-1.913235,-0.617170,-0.367936,-0.952737,2.065924,-2.931635,-4.936853,...,-1.148173,-1.024835,0.321771,-0.357173,1.042426,0.436647,-0.193977,0.347655,0.208514,-0.586731,-0.691006,1.185702,0.409145,-2.550551,0.616470,0.480213,0.349182,-0.342722,1.907002,1.102293,0.458174,-0.909848,-0.131525,-1.388471,0.626571,-0.925222,2.790391,0.539964,-1.875945,0.667377,-0.392362,-0.462895,0.628174,1.924847,1.199006,-1.113347,-1.724282,-1.388951,-1.468666,CC1(C)[N:19]([C:24]([CH:25]([F:26])[F:27])=[O:...
[CH2:1]([CH2:2][CH2:3][CH

In [36]:
bad_mol = set([np.nan, 'F[Cl:1](=[O:2])(=[O:3])=[O:4]', 'O=[N+:1]([O-])[C:4]1=[CH:5][C:6]([Cl:10](=[O:11])(=[O:12])=[O:13])=[CH:7][CH:8]=[CH:9]1', '[Cl:2](=[O:3])(=[O:4])(=[O:5])[C:6]1=[CH:12][CH:11]=[CH:10][C:8]([NH2:9])=[CH:7]1', '[Cl:1](=[O:2])(=[O:3])(=[O:4])[C:5]1=[CH:10][CH:9]=[CH:8][C:7]([N:11]=[C:12]=[O:13])=[CH:6]1', '[H]1[BH2][H][BH2]1', 'O=S=Cl', 'O=Cl(=O)(=O)[F:30]', 'O=Cl(=O)(=O)[F:19]', 'O=Cl(=O)(=O)[F:40]', 'Br[Br-]Br', 'Br[Br-][Br:32]', 'Br[Br-][Br:22]', 'Br[Br-][Br:21]', 'O=Cl(=O)(=O)F', '[NH4+]=S', 'C=Cl', 'Br[Br-][Br:33]', 'O[ClH](O)(=O)=O', 'Br[Br-][Br:7]', 'O=C([O-])[Cl-][CH2:18][C:17]1=[CH:16][CH:15]=[C:14]([N+:23](=[O:24])[O-:25])[CH:13]=[CH:12]1', 'FBr(F)F', 'Br[Br-][Br:26]', 'Br[Br-][Br:24]', 'Cl[CH:8]([CH2:7][N:1]1[CH2:2][CH2:3][CH2:4][CH2:5][CH2:6]1)[CH2:9][ClH:10][C:11]([C:12]1=[CH:13][CH:14]=[CH:15][CH:16]=[CH:17]1)=[N:18][OH:19]', 'CCCC[B+3]CCCC', 'CC(C)(C)[ClH:21]O', '[NH4+]=[S:19]', 'Br[Br-][Br:23]', 'Br[Br-][Br:30]', '[NH4+]=[S:20]'])

In [37]:
good_mol = set(df_react.mol.unique()) - bad_mol

In [38]:
pca_df = pca_df.loc[good_mol].reset_index(drop = True)
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,mol
0,-14.084100,-6.431431,-4.935615,-2.673653,-1.375062,2.659896,-11.681773,3.652557,4.009928,-7.992149,-6.859528,3.561865,-2.982639,0.829621,-7.257622,-1.286764,3.070447,-4.674988,-6.934114,-5.579943,2.891305,1.529625,1.780309,-0.714171,0.933368,-2.512935,4.322643,-1.836477,3.583496,-0.882071,4.612514,0.597937,-2.173106,-2.268414,-3.171392,-2.436799,1.394461,-1.857973,7.115027,1.978605,...,1.661911,-1.785418,0.210754,-0.611111,-0.685022,0.793490,1.194745,-2.858156,0.576370,-1.470182,-0.026424,-1.931826,-0.780735,-1.330317,2.083672,1.791409,-0.612329,0.535983,-0.065099,0.609390,2.032779,0.685925,-1.043547,-0.341498,1.147942,0.284077,0.950963,-0.791153,-1.432655,0.578559,-0.381402,-0.195801,0.055609,-0.732974,1.317147,-1.166164,-0.518661,-0.050496,-1.974376,[CH3:8][C:9]1=[C:18]2[CH:12]([CH2:11][CH2:10]1...
1,-16.206959,18.492855,-12.426883,6.728848,-7.346689,-11.029277,7.517149,-1.694062,2.083994,0.293413,3.096449,8.577111,1.731069,2.094786,-2.354896,-3.159351,4.350002,-1.542087,2.004511,-0.919538,-5.471645,2.862835,3.212252,2.001296,-2.946923,1.939624,-0.742364,-1.512017,1.999144,0.325971,-5.855859,-0.979691,-2.168102,0.158414,7.568893,-0.559834,4.468496,-1.963622,1.484757,2.007423,...,0.055959,-0.293954,0.172788,0.523795,0.555376,0.605863,0.221989,0.709318,-0.305361,-0.348749,0.211709,0.195896,-0.513286,-0.214873,-0.509005,0.944317,-0.710802,0.353201,-0.518531,-0.115805,-0.356334,0.082347,-0.683549,-0.662951,0.191460,-0.566184,-0.951033,0.311831,-0.142521,-0.112069,-0.726212,0.668797,-0.292424,-0.435144,0.380344,-0.945300,0.428932,-1.359218,0.202773,N[C:23]1=[CH:24][C:19]([CH3:18])=[C:20]([OH:27...
2,-16.052987,-14.736020,-0.808388,2.487337,2.589232,14.386850,-2.756294,-5.965177,-0.411463,6.099654,6.270726,4.547475,-3.752128,3.746920,-1.909975,-4.470509,-0.850142,-1.973741,-2.487573,3.300886,-0.376447,4.675720,-0.484731,1.177204,-0.636461,-2.956061,-3.196972,-4.794305,-1.258497,6.097856,-3.424421,-0.626680,-1.391789,5.814161,1.889270,1.149901,2.401756,-0.141911,2.322222,-0.560355,...,-0.557289,-0.121241,0.028165,0.295330,0.563957,-0.683418,0.002752,-0.021846,0.038178,-0.358263,-1.080343,-0.123714,-0.148762,0.034567,-0.687170,-0.036909,-0.275407,0.715975,0.300185,0.171027,0.174117,0.069708,-0.841034,-0.809151,-0.157551,0.007009,-0.634180,-0.814188,0.156816,0.431091,-0.117554,-0.153414,0.037138,0.423423,-0.147830,-0.397867,-0.001094,-0.589714,-0.500672,[C:12]1([OH:22])=[CH:13][CH:14]=[CH:15][C:16]2...
3,-22.530960,-11.701595,-4.264212,-0.793151,0.965996,5.395092,8.354345,-5.091061,-3.838224,-2.295425,-3.905298,4.858011,-4.557454,-5.930549,1.949468,-0.120803,-6.983976,-2.707264,2.147617,-0.257558,1.812946,-1.846528,2.265548,-0.067203,1.765481,-0.883106,-1.141405,-3.418009,1.983112,-1.167568,3.279974,-0.470955,-1.940037,-2.462626,0.054783,-0.679576,3.548811,-0.944565,-0.766066,0.687710,...,-0.008992,-0.372141,-0.370989,-0.462039,0.674231,-0.182463,0.215458,-0.175858,0.164048,-0.137560,-0.416910,-0.033104,-0.277028,-0.256875,-0.809890,0.222593,0.168552,0.155307,-0.218870,-0.128707,0.316159,0.421025,0.146336,-0.185507,-0.156137,1.034258,-0.242655,-0.080832,-0.035351,0.507398,-0.301889,-0.684695,0.435544,0.095750,0.058935,0.400338,-0.553818,-0.060171,-0.122342,[N:1]1=[CH:2][CH:3]=[C:4]([CH3:7])[CH:5]=[CH:6]1
4,-8.327445,14.147006,-9.897398,7.799134,-6.457722,15.401175,-11.345430,-2.823543,12.050472,5.433036,6.133290,-4.721481,2.175645,0.505212,2.984909,1.513167,-0.193040,3.305774,-3.653939,0.720718,4.221675,2.443199,-2.785591,-1.178100,2.301364,0.138701,-1.081113,-3.394896,-1.877455,-2.497652,1.943128,-1.587099,2.667676,-1.646622,1.561850,2.823773,-2.042026,1.899062,-2.547701,0.780747,...,0.787385,-0.103329,-1.190883,-0.811419,

In [39]:
tqdm.pandas()

In [40]:
def grouper_mean(dataset):
    merged = (pd.merge(dataset, pca_df, left_on='mol', right_on='mol', how='left')
              .drop(['mol'], axis=1)
              .fillna(0))
    print('Терпение')
    mean = merged.groupby('index').mean()
    
    return mean

In [41]:
result_dataframe = grouper_mean(df_react)
result_dataframe.shape

Терпение


(10000, 1922)

In [42]:
result_dataframe.to_csv('dataframe_10k.tsv', sep = '\t', index = 0)

In [49]:
# itog = grouper_mean(df_react[df_react['index'] == 0]).drop(0.0)

# for batch in tqdm(range(1, 9)):
#     batch*=100000
    
#     mean_df = grouper_mean(df_react[(df_react['index'] >= batch-100000) 
#                                              & (df_react['index'] < batch)])
    
#     itog = pd.concat([itog, mean_df], axis = 0)

In [ ]:
# df_res.to_excel('feature_mol.xlsx')

[0;31mKernelOutOfMemory[0m: Kernel ran out of memory and has been restarted. If the restart fails, restart the kernel from the Kernel menu.
If the error persists, try choosing a different configuration or optimizing your code.

In [28]:
# def prepare_generator(col):
#     for col_val in tqdm(col):
#         merged = pd.merge(reaction_batch[col_val], df_res, left_on=col_val, right_on='mol', how='left')
#         dropped = merged.drop([col_val, 'mol', 'prepare_mol'], axis=1).fillna(0)
        
#         yield dropped


# for i in tqdm(range(1, 9)):
#     i *= 100000

#     reaction_batch = reaction.iloc[i-100000: i]
#     result_tensors = []
#     res = list(prepare_generator(reaction.columns[:-1]))  # Преобразуем генератор в список
#     res = np.transpose(np.array(res), (1, 0, 2)).astype('float16')

#     with h5py.File(f'./result_batch/result.hdf5', 'a') as f:
#         dset = f.create_dataset(f"default_{i}", data=res)

#     # final_tensor = torch.cat(result_tensors, dim=1)
#     # torch.save(tensor_result, f'./result_tensor_{i}.pt')

In [ ]:
reaction

In [71]:
def prepare_generator(index):
    merged = pd.merge(reaction.iloc[index], df_res, left_on=index, right_on='mol', how='left')
    dropped = merged.drop([index, 'mol', 'prepare_mol'], axis=1).fillna(0).astype('float32')
    mean = dropped.mean(axis = 0)
    
    return list(mean)


In [72]:
res = [prepare_generator(index) for index in tqdm(range(reaction.shape[0]))]

  0%|          | 39/774932 [01:05<362:50:42,  1.69s/it]

KeyboardInterrupt: 

In [70]:
pd.DataFrame(res, columns = df_res.columns[2:])

,rdkit_2d_MaxAbsEStateIndex,rdkit_2d_MaxEStateIndex,rdkit_2d_MinAbsEStateIndex,rdkit_2d_MinEStateIndex,rdkit_2d_qed,rdkit_2d_SPS,rdkit_2d_MolWt,rdkit_2d_HeavyAtomMolWt,rdkit_2d_ExactMolWt,rdkit_2d_NumValenceElectrons,rdkit_2d_NumRadicalElectrons,rdkit_2d_MaxPartialCharge,rdkit_2d_MinPartialCharge,rdkit_2d_MaxAbsPartialCharge,rdkit_2d_MinAbsPartialCharge,rdkit_2d_FpDensityMorgan1,rdkit_2d_FpDensityMorgan2,rdkit_2d_FpDensityMorgan3,rdkit_2d_BCUT2D_MWHI,rdkit_2d_BCUT2D_MWLOW,rdkit_2d_BCUT2D_CHGHI,rdkit_2d_BCUT2D_CHGLO,rdkit_2d_BCUT2D_LOGPHI,rdkit_2d_BCUT2D_LOGPLOW,rdkit_2d_BCUT2D_MRHI,rdkit_2d_BCUT2D_MRLOW,rdkit_2d_AvgIpc,rdkit_2d_BalabanJ,rdkit_2d_BertzCT,rdkit_2d_Chi0,rdkit_2d_Chi0n,rdkit_2d_Chi0v,rdkit_2d_Chi1,rdkit_2d_Chi1n,rdkit_2d_Chi1v,rdkit_2d_Chi2n,rdkit_2d_Chi2v,rdkit_2d_Chi3n,rdkit_2d_Chi3v,rdkit_2d_Chi4n,...,ecpf6_bit2008,ecpf6_bit2009,ecpf6_bit2010,ecpf6_bit2011,ecpf6_bit2012,ecpf6_bit2013,ecpf6_bit2014,ecpf6_bit2015,ecpf6_bit2016,ecpf6_bit2017,ecpf6_bit2018,ecpf6_bit2019,ecpf6_bit2020,ecpf6_bit2021,ecpf6_bit2022,ecpf6_bit2023,ecpf6_bit2024,ecpf6_bit2025,ecpf6_bit2026,ecpf6_bit2027,ecpf6_bit2028,ecpf6_bit2029,ecpf6_bit2030,ecpf6_bit2031,ecpf6_bit2032,ecpf6_bit2033,ecpf6_bit2034,ecpf6_bit2035,ecpf6_bit2036,ecpf6_bit2037,ecpf6_bit2038,ecpf6_bit2039,ecpf6_bit2040,ecpf6_bit2041,ecpf6_bit2042,ecpf6_bit2043,ecpf6_bit2044,ecpf6_bit2045,ecpf6_bit2046,ecpf6_bit2047
0,1.981109,1.981109,0.030927,-0.104885,0.140044,2.469231,45.403202,42.177601,45.317406,16.600000,0.0,0.020220,-0.100913,0.100913,0.049538,0.423077,0.500000,0.573077,8.386853,5.620214,0.358296,-0.395955,0.443234,-0.320008,1.767742,0.602274,0.361373,0.377966,81.034813,2.144095,1.805162,1.968461,1.389473,0.997490,1.096050,0.680608,0.814351,0.471970,0.559565,0.357293,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.968949,1.968949,0.052867,-0.002867,0.181736,2.068000,49.212196,46.087399,49.112995,16.600000,0.0,0.035449,-0.087575,0.287575,0.238704,0.620000,0.700000,0.776000,9.986753,7.233587,0.254022,-0.475676,0.410350,-0.327416,1.850122,0.685853,0.363302,0.379117,81.604828,2.051830,1.892186,2.367796,1.354877,0.962154,1.060714,0.621433,0.755176,0.429888,0.517482,0.319395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.494444,0.494444,0.100000,0.100000,0.234067,-264.000000,11.006200,9.796600,11.008636,3.800000,0.0,0.004664,-0.038145,0.838145,0.804664,0.300000,0.360000,0.380000,2.051008,1.442558,-0.205725,-0.576403,0.242826,-0.129303,0.887975,0.464583,0.176050,0.208333,1.521928,0.353553,0.881403,0.596873,0.250000,0.207735,0.207735,0.131948,0.131948,0.082735,0.082735,0.051031,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.754713,1.754713,0.338389,0.225889,0.137842,3.079121,35.342201,33.023800,35.317394,13.600000,0.0,0.035692,-0.085450,0.085450,0.048146,0.286081,0.423626,0.532601,4.532063,3.137803,0.565290,-0.554585,0.576165,-0.541718,1.659976,0.233356,0.563735,0.871596,62.528877,1.894095,1.501400,1.501400,1.246923,0.840741,0.840741,0.577130,0.577130,0.358757,0.358757,0.261259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.655393,1.655393,0.295139,0.254889,0.187247,3.107692,44.403900,42.085503,44.095848,11.600000,0.0,-0.019631,-0.077169,0.077169,0.062671,0.448718,0.502564,0.533333,14.785851,11.681713,0.441785,-0.512846,0.566656,-0.424985,2.226704,1.122213,0.472786,0.508844,22.205969,1.481999,1.165774,1.634159,0.828715,0.550616,1.028949,0.518944,0.599756,0.232327,0.232327,0.119932,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,